In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

api_token = os.getenv('API_TOKEN')
pgrescreds = os.getenv('DB_PASSWORD')

In [2]:
import requests 
import pandas as pd
import numpy as np
import sys
from apscheduler.schedulers.blocking import BlockingScheduler
from sqlalchemy import create_engine
from datetime import date,datetime,timedelta
import time

In [3]:
engine = create_engine(pgrescreds)

In [6]:
sdata_base_url = 'https://api.stockdata.org/v1/data/quote?symbols='
sdata_intraday_url = 'https://api.stockdata.org/v1/data/intraday?symbols='
tickers = ['META','AAPL','NVDA','GOOGL']
interval = "hour"

In [1]:
def hourly_stock_data(url,interval,ticker,token,date):
    response = requests.get(f'{url}{ticker}&interval={interval}&date_from={date}&api_token={token}')
    print(response.status_code)
    data = response.json()
    if not data['data']:
        print(f'No data available for {ticker} on {date}')
        return None
        
    df = pd.json_normalize(data['data'])
    df.columns = [col.replace('data.', '') for col in df.columns]  
    return df


def data_preprocessing(df,ticker):
    df['date'] = pd.to_datetime(df['date'], utc=True)  
    df['date'] = df['date'].dt.tz_convert('US/Eastern').dt.tz_localize(None)
    df['date'] = df['date'].dt.strftime('%m-%d-%y %H:%M')
    return df
    
def update_db(df,ticker):
    df.to_sql(ticker, engine, if_exists = 'replace', index = False)
    print('DB Updated!')


def run_pipeline(ticker):
    spec_date = date.today() - timedelta(days=4)
    df = hourly_stock_data(sdata_intraday_url,interval,ticker,api_token,spec_date)

    if df is None:
        print(f'Skipping pipeline for {ticker}, no data returned.')
        return
    
    df = data_preprocessing(df,ticker)
    update_db(df,ticker)

scheduler = BlockingScheduler()

for ticker in tickers:
    scheduler.add_job(run_pipeline, 'cron', day_of_week='mon-fri',
                     hour='9-16', minute = '0', args=[ticker])
    time.sleep(1)
    
scheduler.start()
    
    
    



NameError: name 'BlockingScheduler' is not defined

In [ ]:
# stock interval still a bit messy, might want to pull at a different interval

In [ ]:
# would like to clean the data series to where time is shown in est if possible 

In [ ]:
# what datatype is date series?

In [ ]:
# likely a string? clean then type cast to int for easier est conversion

In [ ]:
# whats Next? grab more data based on ticker. realtime data? create db, create visualizations